In [3]:
import sys
import time
import random

# Generate random grid
def generate_random_grid(width, height, alive):

    if width <= 0 or height <= 0:
        raise ValueError("Width and height must be positive")
    
    grid = []
    if width > 0 and height > 0:   
        for y in range(height):
            grid.append([0] * width)

    if alive < 0 or alive > 1.0:
        raise ValueError("Invalid alive percentage.")
    
    m = round(width * height * alive)
    skip = {}

    while m > 0:
        x, y = random.randrange(width), random.randrange(height)
        idx = y * width + x
        if idx not in skip:
            grid[y][x] = 1
            skip[idx] = True
            m -= 1

    return grid

# save the grid to the text file
def save_grid_to_file(output_file, grid):
    for row in grid:
        for cell in row:
            output_file.write(str(cell)+" ")
        output_file.write('\n')

# to calculate the alive neighbors
def count_live_neighbors(grid, x, y, width, height):
    # the neighbor matrix
    neighbors = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    # initialize the count
    count = 0
    # loop all possible neighbors
    for dx, dy in neighbors:
        nx, ny = x + dx, y + dy
        # if its neighbor doesn't go beyond the boundary and is alive 
        if 0 <= nx < width and 0 <= ny < height and grid[ny][nx]==1:
            count += 1

    return count

# update the gird
def update_grid(grid, width, height):
    new_grid = [[0] * width for _ in range(height)]

    #apply the rule
    for y in range(height):
        for x in range(width):
            live_neighbors = count_live_neighbors(grid, x, y, width, height)

            if grid[y][x] == 1:
                # Live cell
                if live_neighbors == 2 or live_neighbors == 3:
                    new_grid[y][x] = 1
            else:
                # Dead cell
                if live_neighbors == 3:
                    new_grid[y][x] = 1

    return new_grid

# run the game
def run_game(output_name, generations, width, height, alive):
    grid = generate_random_grid(width, height, alive)

    output_grid_path =output_name.split(".")[0]+"_original_grid.txt"
    with open(output_grid_path, 'w') as output_file:
        save_grid_to_file(output_file, grid)

    start = time.time()
    for _ in range(generations):
        grid = update_grid(grid, width, height)
        print("{} seconds run for {} generations.".format(round(time.time() - start, 5), _))

    with open(output_name, 'w') as output_file:
        save_grid_to_file(output_file, grid)

## main function
def main():
    run_game("test_data/g1000.txt", 1, 1000, 1000, 0.1)

if __name__ == "__main__":
    main()

3.26891 seconds run for 0 generations.


In [ ]:
import numpy as np
import dask.array as da

In [ ]:
# NumPy array
a_np = np.random.rand(5,5)
print(a_np)
print(a_np[0:1, 0:2])

In [ ]:
a_np_sum = a_np[:5].sum() + a_np[5:].sum()
a_np_sum

In [ ]:
a_da = da.from_array(a_np, chunks=(2,2))
a_da


In [ ]:
import numpy as np
import dask.array as da

def mean_square(arr):
    arr =arr +1
    return arr

# Example usage with NumPy array
a_np = np.random.rand(5, 5)

# Example usage with Dask array
a_da = da.from_array(a_np, chunks=(2, 2))
a_da_mean_square = mean_square(a_da)
print(a_da_mean_square.compute())

In [ ]:
a_np = np.arange(1, 26).reshape((5, 5))
a_np

In [ ]:
import numpy as np
import dask.array as da

def f(arr):
    # Create a new array of zeros with the same shape as the input array
    result = np.zeros_like(arr)
    
    # Iterate over each element in the input array
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            # Calculate the sum of the surrounding elements
            s = 0
            for k in range(max(0, i-1), min(arr.shape[0], i+2)):
                for l in range(max(0, j-1), min(arr.shape[1], j+2)):
                    if k != i or l != j:
                        s += arr[k, l]
            
            # If the sum is greater than 20, set the element to 0
            if s > 30:
                result[i, j] = 0
            else:
                result[i, j] = 1
    
    return result

# Example usage with NumPy array
a_np = np.arange(1, 26).reshape((5, 5))
print(f(a_np))

# Example usage with Dask array
a_da = da.from_array(a_np, chunks=(1, 5))
a_da_mean_square = f(a_da)
print(a_da_mean_square.compute())

In [129]:
import numpy as np
import dask.array as da

def f(arr):
    # Create a new array of zeros with the same shape as the input array   
    #return arr + np.roll(arr, 1)
    return arr+arr.size

# Example usage with NumPy array
# Create a NumPy array with integers from 1 to 25
a_np = np.arange(1, 26)
a_np = a_np.reshape((5, 5))
print(a_np)
print(f(a_np))

# Example usage with Dask array
a_da = da.from_array(a_np, chunks=(2, 2))
#a_da_n = da.map_blocks(f, a_da)
#a_da_n=f(a_da)
a_da_n = a_da.map_overlap(f, depth=1, boundary='none')
print(a_da_n.compute())

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[26 27 28 29 30]
 [31 32 33 34 35]
 [36 37 38 39 40]
 [41 42 43 44 45]
 [46 47 48 49 50]]
[[10 11 15 16 11]
 [15 16 20 21 16]
 [23 24 29 30 23]
 [28 29 34 35 28]
 [27 28 31 32 29]]


2023-10-17 02:48:40,422 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 3 memory: 23 MB fds: 27>>
Traceback (most recent call last):
  File "/Users/yokosukamori/Library/CloudStorage/OneDrive-UniversityofTwente/Geo-Bigdata-Processing/Game_of_Life/Rispy/lib/python3.9/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/Users/yokosukamori/Library/CloudStorage/OneDrive-UniversityofTwente/Geo-Bigdata-Processing/Game_of_Life/Rispy/lib/python3.9/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/yokosukamori/Library/CloudStorage/OneDrive-UniversityofTwente/Geo-Bigdata-Processing/Game_of_Life/Rispy/lib/python3.9/site-packages/psutil/__init__.py", line 2119, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
2023-10-17 13:06:23,443 - tornado.application - ERROR - Exception in

In [ ]:
import numpy as np
import dask.array as da
from dask import delayed
from dask.distributed import Client
import matplotlib.pyplot as plt

# 创建 Dask 客户端
client = Client()

# 创建随机初始状态
initial_state = np.random.randint(0, 2, size=(100, 100))

# 转换为 Dask 数组
initial_dask_state = da.from_array(initial_state, chunks=(25, 25))

# 定义游戏规则的函数
def game_of_life_chunk(chunk):
    neighbor_count = sum(chunk[-1:, :], chunk[:1, :], chunk[:, -1:], chunk[:, :1], chunk[-1:, -1:], chunk[-1:, :1], chunk[:1, -1:], chunk[:1, :1])
    new_chunk = (chunk == 1) & ((neighbor_count == 2) | (neighbor_count == 3)) | (chunk == 0) & (neighbor_count == 3)
    return new_chunk

# 迭代更新状态
def iterate_state(state, iterations):
    for _ in range(iterations):
        new_state = da.blockwise(game_of_life_chunk, 'ij', state, 'ij', dtype=bool, new_axes={0: 100, 1: 100})
        state = new_state.compute()
    return state

# 迭代更新状态并绘制
final_state = iterate_state(initial_dask_state, iterations=100)
plt.imshow(final_state, cmap='binary')
plt.show()
